# Loading of data

Data was loaded from 3 sources

1. .csv file downloaded manually from https://d17h27t6h515a5.cloudfront.net/topher/2017/August/59a4e958_twitter-archive-enhanced/twitter-archive-enhanced.csv
2. .tsv file downloaded programmatically from https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv
3. .txt file gathered through Twitter API using the Tweepy module

The following code was used to load and or gather the data 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import statsmodels.api as sm
%matplotlib inline

In [3]:
#load csv file
twitter_archive = pd.read_csv("twitter-archive-enhanced-2.csv")

In [4]:
#load additional modules
import requests
import os
# download the image predictions file
# commented out for repeat running of code
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
#download dog breed predictions file and store in base directory
with open('predictions.tsv',mode = 'wb') as file:
    file.write(response.content)
# open file
predictions = pd.read_csv('predictions.tsv', delimiter = '\t' )
predictions.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [ ]:
# import json module
import json
from timeit import default_timer as timer
# gather tweet data through twitter API and store in .txt file
count = 0
fails_dict = {}
start = timer()
tweet_ids = predictions.tweet_id.values

with open ('tweet_json3.txt', 'w', encoding = "utf-8") as outfile:
    for tweet_id in tweet_ids:
        count += 1
        print(str(count) + ": " + str(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode= 'extended', wait_on_rate_limit = True, wait_on_rate_limit_notify = True)
            print("succes")
            json.dump(tweet._json, outfile)
            outfile.write("\n")
        except tweepy.TweepError as e:
            print("Fail")
            fails_dict[tweet_id] = e
            pass
end = timer()
#load .txt file and store in Data Frame
tweets = []
with open( 'tweet_json3.txt', 'r', encoding = "utf-8") as file:
    for line in file:
        tweet = json.loads(line)
        tweet_id = tweet['id']
        retweet_count = tweet['retweet_count']
        favorite_count = tweet['favorite_count']
        tweets.append({'tweet_id': tweet_id, 'retweet_count': retweet_count, 'favorite_count': favorite_count})
api_twitter_counts = pd.DataFrame(data= tweets)

# Assesment

Each individual file was visually and programmatically assessed 

## Visual assesments

The `twitter_archive_enhanced.csv` file was visually assesed but no issues where identified from this assesment. The `predictions.tsv` was visually assesed, from this assesment it was concluded that several rows contained no correct predictions of dog breeds. The `tweet_json.txt` file was not visually assesed.
twitter_archive_clean.info()

## programmatic assesment
Programmatic assesment contains of initially utlising the `pd.info()` command to check for `NaN` values. Furthermore checks are performed on different columns to see if they contain expected data forms using regex functions like:

In [ ]:
#check tweet_id to see if the tweet_id column contains a sequence of digits or not
twitter_archive_clean[~twitter_archive_clean['tweet_id'].astype(str).str.contains(r'\d{18}')]

Further columns are also checked for the following expected values in the same manner:
* timestamp column is expected to contain data in the same time format for each rows
* rating numerator and rating denominator are expected to be one or two digit numbers
* The name column is expected to be a single word
Further checks are performed to see if columns contain unexpected or unwanted values that are not found by the regex operation
* the name column is not expected to contain the string 'None' 
* the "doggo", "floofer", "pupper" and "puppo" columns are expected to contain only these respective strings and not the string "none"
Checking for all the unique values in different columns can give us an idea of further data quality issues
* The source column may contain many unique items
* the "doggo", "floofer", "pupper" and "puppo" columns are expected to contain no unique items besides the respective strings and the string "none"4
* the different favorite and retweet id columns are expected to only contain id's or NaN values

The programmatic assesment continued with the assesment of the predictions file
* Confirming visual assesment that some tweets contain no accurate prediction of dog breed

## Issues
Through the above described assesment the following issues are identified:
### Data tidyness
#### twitter archive .csv file
     1. `doggo`, `floofer`, `pupper` and `puppo` are mutually exclusive, should be a single column

#### dog race predictions .tsv file
     2. Correct predictions for dog breed are not presented in a single column
     3.the predictions are required for final analysis, and therefore should be added to the twitter archive DataFrame
#### tweet_json.txt file
     4. the api_twitter_counts dataframe needs to be added to the twitter_archive_clean dataframe
### Data quality
#### twitter archive .csv file
     1.The `name` column contains 745 rows with the string "None" instead of a name
     2.The new single `doggo`,`floofer`,`pupper`,`puppo` column contains rows with the string "None" 
     3.The alternate id columns contain mostly NaN values 
     4.The `retweeted_status_timestamp` column contains mostly NaN values
     5.The alternate id columns data when non NaN values are float, should be integer or string
     6.`timestamp` should be in datatime format
     10.After creating the `dog_type` column to solve tidyness issue 1 and quality issue 2, it contains a lot of NaN values

#### dog race predictions .tsv file
     7.The entire dataset contains 2075 entries as opposed to 2256 entries for twitter archive
     8.324 tweets have no positive recognition of dog breed, resulting in missing data
#### tweet_json file
     9.resulting dataframe has 2075 entries as opposed to 2256 entries for twitter archive

# Cleaning

Cleaning commences with the completeness issues identified. These being the **quality issues 3 and 4**. These issues are cleaned by first removing the rows with non-NaN values. These rows are replies or retweets and can be removed, as per the assignment description

In [ ]:
#check total number of rows to be deleted
len(twitter_archive_clean.query('in_reply_to_status_id.notnull() or in_reply_to_user_id.notnull() or retweeted_status_id.notnull() or retweeted_status_user_id.notnull()'))

In [ ]:
#find rows with no nan values
lstid1 = twitter_archive_clean[twitter_archive_clean.in_reply_to_status_id.notnull()].index.values.tolist()
lstid2 = twitter_archive_clean[twitter_archive_clean.in_reply_to_user_id.notnull()].index.values.tolist()
lstid3 = twitter_archive_clean[twitter_archive_clean.retweeted_status_id.notnull()].index.values.tolist()
lstid4 = twitter_archive_clean[twitter_archive_clean.retweeted_status_user_id.notnull()].index.values.tolist()
# combine lists
lst_id_temp= lstid1 + lstid2 + lstid3 + lstid4
# remove duplicates by converting to dictionary and back to list
dic = dict.fromkeys(lst_id_temp)
#convert back to list
lst_id = list(dic)
#check length of list
len(lst_id)

In [ ]:
#drop the rows with retweeted tweets
twitter_archive_clean.drop(lst_id, inplace=True)

After removing these rows, the columns 'in_reply_to_status_id','in_reply_to_user_id','retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp' can be dropped as these rows play  no role in further analysis

In [ ]:
twitter_archive_clean.drop(columns = ['in_reply_to_status_id','in_reply_to_user_id','retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp'], inplace = True)

The cleaning effort continuous with tidyness issues
## Tidyness
The combining of the `doggo`, `floofer`, `pupper` and `puppo` columns is done by replacing the string `None` with `NaN`. Then nesting a fillna command for all four columns and filling a copy of the `doggo` column with `floofer`, `pupper` and `puppo` values respectively

In [ ]:
twitter_archive_clean['dog_type'] =twitter_archive_clean["doggo"].fillna(twitter_archive_clean["floofer"].fillna(twitter_archive_clean["pupper"].fillna(twitter_archive_clean["puppo"])))

This solves tidyness **issue 1** and also **quality issue 2**.

**tidyness issue 2 and 3** are cleaned by sourcing data from several columns in the `predictions` dataframe, and copying this in a new column called `breed`. This column is subsequently added to the `twitter_archive_clean` DataFrame. The desired columns from the `predictions` dataframe is merged to the `twitter_archive_clean` dataframe using an left join. This way only additional information on the tweets win the `twitter_archive_clean` dataframe is added. This solves **quality issue 7**


In [ ]:
#solution to tidyness issue 2
comparison = np.where(predictions["p1_dog"] == True, predictions['p1'], np.where(predictions['p2_dog']== True, predictions['p2'], np.where(predictions['p3_dog']== True, predictions['p3'],"unknown")))
# add column to the original dataframe
predictions["breed"] = comparison

The retweet and favorite count data is added to the `twitter_archive_clean` dataframe by an inner join, as to only retain tweets that have information in both datasets. This solves **tidyness issue 4** and **quality issue 9**

## Quality

The remaining quality issues are as follows:
#### twitter archive .csv file
     1.The `name` column contains 745 rows with the string "None" instead of a name
     6.`timestamp` should be in datatime format
     10.After creating the `dog_type` column to solve tidyness issue 1 and quality issue 2, it contains a lot of NaN values
#### dog race predictions .tsv file
     8.324 tweets have no positive recognition of dog breed, resulting in missing data

**quality issue 1** is cleaned by removal of the `name` column. As no analysis is planned using this information. **quality issue 6** is resolved by changing the datatype for the `timestamp` column by:

In [ ]:
twitter_archive_clean = twitter_archive_clean.astype({'timestamp' : 'datetime64'})

**quality issue 10** is addressed by scanning for additional occurences of the strings `doggo`, `floofer`, `pupper` and `puppo` in the `text` column

In [ ]:
twitter_archive_clean['dog_type2'] = twitter_archive_clean['text'].str.extract('(doggo|floofer|pupper|puppo)', expand = True)

These occurences are added to a new column `dog_type2` as the missing values are filled in as `NaN`, the same tactic as before can be utilised to fill in the additional data to the `dog_type` column

In [ ]:
twitter_archive_clean['dog_type']= twitter_archive_clean.dog_type.fillna(twitter_archive_clean.dog_type2)

**quality issue 8** is accepted, since there is no way to programmatically asses the missing data within the scope of this project. Since over a 1000 tweets do have a accurate prediction of dog breed, future analysis can be performed.

The cleaned data is stored as a .csv file as `twitter_archive_master.csv`
